Cleaning Test Data and merging the 4 Test Files

In [0]:
import pandas as pd
import numpy as np
import collections
import glob

In [0]:
# Reading all 4 Test Files
t1 = pd.read_csv("test1.csv")
t2 = pd.read_csv("test2.csv")
t3 = pd.read_csv("test3.csv")
t4 = pd.read_csv("test4.csv")

In [0]:
# Merging all Test Files and Sorting them on the basis of CUST_CODE,SHOP_WEEK and BASKET_ID.
test = pd.concat([t1,t2,t3,t4], axis = 0)
test = test.sort_values(['CUST_CODE','SHOP_WEEK','BASKET_ID'], ascending= [True, True,True])

In [5]:
# Keeping the products that were roeorderd
reordered_test = test[(test['reorder'] == 1)]
reordered_test = reordered_test.reset_index(drop = True)
reordered_test.head()

,SHOP_WEEK,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x,reorder,BASKET_NUM_y
0,200619,PRD0900219,CUST0000000001,994101300170532,14,1,14
1,200619,PRD0900330,CUST0000000001,994101300170532,14,1,14
2,200619,PRD0900440,CUST0000000001,994101300170532,14,1,14
3,200619,PRD0900830,CUST0000000001,994101300170532,14,1,14
4,200619,PRD0900965,CUST0000000001,994101300170532,14,1,14


In [9]:
# 4 test files were made for each quarter but only 1 order can be used as a test order.
# Out of the 4 ttest orders, the last one is kept if it has reordered products
# If last order does not have reordered products, test files from previous quarter is used.
last_basket = reordered_test.groupby(['CUST_CODE']).apply(lambda x: (x.sort_values(['CUST_CODE','SHOP_WEEK','BASKET_NUM_x'], 
                                          ascending=[True, False, False]).head(1)))
last_basket = last_basket[["SHOP_WEEK","CUST_CODE","BASKET_ID","BASKET_NUM_x"]]
last_basket = last_basket.reset_index(drop = True)
last_basket

,SHOP_WEEK,CUST_CODE,BASKET_ID,BASKET_NUM_x
0,200706,CUST0000000001,994105200166936,15
1,200705,CUST0000000009,994105100165297,7
2,200705,CUST0000000010,994105100165298,21
3,200642,CUST0000000012,994103600156346,4
4,200705,CUST0000000020,994105100165306,5
...,...,...,...,...
290649,200706,CUST0000999987,994105200800670,11
290650,200644,CUST0000999991,994103800558001,9
290651,200642,CUST0000999993,994103600811346,3
290652,200632,CUST0000999996,994102600796913,11


In [0]:
# Creating a flag column for the actual test orders.
last_basket["FLAG"] = 1

In [20]:
# Keeping only the last basket by merging the reordered_test with last_basket data and keeping rows with flag 1
final_test = pd.merge(reordered_test, last_basket, on = ["SHOP_WEEK","CUST_CODE","BASKET_ID","BASKET_NUM_x"], how = 'left')
final_test = final_test[final_test['FLAG'] == 1]
final_test

,SHOP_WEEK,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x,reorder,BASKET_NUM_y,FLAG
51,200706,PRD0900290,CUST0000000001,994105200166936,15,1,15,1.0
52,200706,PRD0900330,CUST0000000001,994105200166936,15,1,15,1.0
53,200706,PRD0900440,CUST0000000001,994105200166936,15,1,15,1.0
54,200706,PRD0900890,CUST0000000001,994105200166936,15,1,15,1.0
55,200706,PRD0901871,CUST0000000001,994105200166936,15,1,15,1.0
...,...,...,...,...,...,...,...,...
3722664,200642,PRD0903678,CUST0000999993,994103600811346,3,1,3,1.0
3722665,200632,PRD0904335,CUST0000999996,994102600796913,11,1,11,1.0
3722666,200632,PRD0903735,CUST0000999996,994102600796913,11,1,11,1.0
3722671,200706,PRD0902087,CUST0000999999,994105200800673,9,1,9,1.0


In [23]:
final_test = final_test.reset_index(drop = True)
final_test = final_test.drop_duplicates()
final_test

,SHOP_WEEK,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x,reorder,BASKET_NUM_y,FLAG
0,200706,PRD0900290,CUST0000000001,994105200166936,15,1,15,1.0
1,200706,PRD0900330,CUST0000000001,994105200166936,15,1,15,1.0
2,200706,PRD0900440,CUST0000000001,994105200166936,15,1,15,1.0
3,200706,PRD0900890,CUST0000000001,994105200166936,15,1,15,1.0
4,200706,PRD0901871,CUST0000000001,994105200166936,15,1,15,1.0
...,...,...,...,...,...,...,...,...
1241980,200642,PRD0903678,CUST0000999993,994103600811346,3,1,3,1.0
1241981,200632,PRD0904335,CUST0000999996,994102600796913,11,1,11,1.0
1241982,200632,PRD0903735,CUST0000999996,994102600796913,11,1,11,1.0
1241983,200706,PRD0902087,CUST0000999999,994105200800673,9,1,9,1.0


In [25]:
# Generating a list of customers for whome test data with reordered products is available.
cust_list = final_test["CUST_CODE"].unique()
print(len(cust_list))
cust_list

290654


array(['CUST0000000001', 'CUST0000000009', 'CUST0000000010', ...,
       'CUST0000999993', 'CUST0000999996', 'CUST0000999999'], dtype=object)

In [26]:
cust = pd.DataFrame(cust_list, columns =['CUST_CODE'])
cust['FLAG'] = 1
cust = cust.sort_values('CUST_CODE', ascending=True)
cust = cust.reset_index(drop = True)
cust.head()

,CUST_CODE,FLAG
0,CUST0000000001,1
1,CUST0000000009,1
2,CUST0000000010,1
3,CUST0000000012,1
4,CUST0000000020,1


In [0]:
# Exporting the test data
final_test.to_csv("final_test.csv", index = False)
# Exporting the last basket of each customer
last_basket.to_csv("last_basket.csv")
# Exporting the list of customer with available test data
cust.to_csv("cust.csv", index = False)